In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install networkx
# !pip install requests
# !pip install openai
# !pip install nest_asyncio
# !pip install tqdm
# !pip install Jinja2

# LLM으로 알파 생성하기
* LLM이 발달하면서 이제 mini 등 가벼운 모델로도 알파를 꽤나 쉽게 만들 수 있는 환경이 되었습니다. 
* 오늘 돌리면서 한 쿼리를 6번 정도 날렸는데 $0.4 정도가 나왔습니다. 
* 한 번 알파 100개 만드는 시도를 할 때 약 100원 정도 들어간다고 생각하시면 될 것 같습니다.
* 사양이 괜찮으신 분들은 (또는 최신 맥북을 쓰신다면) ollama로 로컬 LLM도 한번 써보세요! 생각보다 괜찮습니다 성능이

# 데이터필드 정보 저장하기
USA TOP3000 기준 30분-1시간 정도 걸리는 작업입니다.

In [4]:
# 처음에만 진행하시면 됩니다.
import AAF as aaf
import ace_lib as ace

# 세팅 
settings = {

    "nan_handling" : "OFF",
    "instrumentType": "EQUITY",      # FIXED
    "delay"          : 1,
    "universe"       : "TOP3000",
    "truncation"     : 0.01,
    "unitHandling"   : "VERIFY",     # FIXED
    "pasteurization" : "ON",
    "region"         : "USA",
    "language"       : "FASTEXPR",   # FIXED

    "decay"          : 0,
    "neutralization" : "MARKET",     # Can be varied
    "visualization"  : False
    
}

s = ace.start_session()
operators = ace.get_operators(s)

aaf.initiate_datafield(s,settings)


Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_oyDktiQWqnBBangyst8m3uaZ3TsP


Region: USA | Universe: TOP3000 | Delay: 1
Total datasets available: 206
Whitelisted datasets to process: 10
Skipped (not in whitelist): 196



Processing datasets: 100%|██████████| 10/10 [01:23<00:00,  8.37s/dataset, Current: socialmedia8]



Creating total.json...
total.json created successfully.

SUMMARY
Successfully processed: 10
Skipped (already exists): 0
Failed (logged to errors.log): 0



# Expression Sanity checker
데이터필드 정보를 이용해서, LLM이 만든 expression이 실제 브레인에서 돌아갈 수 있는지를 체크합니다.

In [5]:
import llm_functions as llm
from parser import *
import ace_lib as ace
import numpy as np
import pandas as pd
import os 
import json
import asyncio



datafields = llm.import_json(f'datafield/1/USA/TOP3000/1_{settings["region"]}_{settings["universe"]}_total.json')
operators = llm.import_json('operators_list.json')

def return_type(node, operators, datafields):
    if node.node_type == 'operator':
        return operators[node.value]['output']

    elif node.node_type == 'number':
        return "NUMBER"

    elif node.node_type == 'datafield':
        if node.value.lower() == 'nan':
            return "MATRIX"
        return datafields[node.value]['type']

    else:
        return "SPECIAL_ARGUMENT"


def check_input(operator_inputs, children_types, _debug = False):
    if _debug: print(operator_inputs)
    while len(operator_inputs) != 0:

        operator_input = operator_inputs.pop(0)
        if _debug: print(f"operator input: {operator_input}, remain inputs: {len(operator_inputs)}" )

        if operator_input == '_SPECIAL_ARGUMENT':
            if len(children_types) == 0:
                pass
            elif children_types[0] == '_SPECIAL_ARGUMENT':
                if _debug: print(operator_input)
                if _debug: print(children_types.pop(0))
            else:
                pass

        else:
            if len(children_types) == 0:
                return False

            children_type = children_types.pop(0)
            if operator_input == "MATRIX" and children_type == "NUMBER":
                pass
            elif operator_input != children_type:
                if _debug: print(operator_input ,children_type, operator_input == children_type)
                return False

    return True


def sanity_checker(exp, _debug=False):

    try:
        exp_tree = tree_node(exp)

        for node in [node for node in exp_tree.collect_all_nodes() if node.node_type == "operator"]:

            if check_input(eval(operators[node.value]['input']), [return_type(x,operators, datafields) for x in node.children]):
                if _debug: print(check_input(eval(operators[node.value]['input']), [return_type(x) for x in node.children]))
            else:
                return False
        if return_type(exp_tree,operators, datafields)=="MATRIX":
            return True

        else:
            return False
    except Exception as e:
        print(f"err: {e}")
        return False

## Sanity check 예시

In [6]:
sanity_checker('bucket(rank(star_si_country_rank), range="0,100,10")') # 이상한 수식, False

c:\Users\adg01\llm_alpha_gen\llm_alpha_gen\parser.py:117: FutureWarning: Possible nested set at position 22
  tokens = re.findall(r'\?|\:|[a-zA-Z_]\w*\(|[[a-zA-Z_]\w*|==|&&|\|\||[-+*/()<>=,]|"(?:\\.[^"\\]*)*"|\'(?:\\.[^\'\\]*)*\'|[0-9]*\.?[0-9]+', expression)


False

In [7]:
sanity_checker('subtract(star_si_shortsqueeze_rank, shrt2_t3m_volatility_rank)') # 정상적인 expression, True

err: 'star_si_shortsqueeze_rank'


False

# 탐색 데이터셋 설정
여기서는 데이터셋 카테고리별로 User Count가 가장 높은 순으로 정렬해서 탐색

In [8]:
s = ace.start_session()

region = "USA"
universe = "TOP3000"

datafields = llm.import_json(f'datafield/1/{region}/{universe}/1_{region}_{universe}_total.json')
operators = llm.import_json('operators_list.json')


datasets = ace.get_datasets(s, region =region, universe = universe)
datasets = datasets.sort_values(['category_id','userCount'],ascending=False)

userCounts = []
DatasetNames = []
datasets = datasets.sort_values(['category_id','userCount'],ascending=False).drop_duplicates(['id'])

for category in datasets.category_id.unique():
    category_datasets = datasets[datasets.category_id == category]
    userCounts.append(category_datasets.userCount.reset_index(drop=True).map(lambda x: int(x) if x>0 else x))
    DatasetNames.append(category_datasets.id.reset_index(drop=True))

DatasetNamesDF = pd.DataFrame(DatasetNames).T
DatasetNamesDF.columns = datasets.category_id.unique()
DatasetNamesDF.to_csv(f'./datasets/1/{region}/{universe}/1_{region}_{universe}_dataset.csv')
datasets_list = [ x for x in sum([list(x) for x in DatasetNamesDF.values],start=[]) if not pd.isna(x)]
datasets_list.remove('pv1')
datasets_dict = {row['id']: row.to_dict() for _, row in datasets.iterrows()}

# 상위 5개 예시
datasets_list[:5]

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_5W4iRqrGkuMrYGYogtWmmY2x2NDJ



['socialmedia12', 'shortinterest43', 'sentiment1', 'risk70', 'other455']

## 시뮬레이션 하기에 앞서...
* 저카운트와 필드카운트에 따라 몇번 expression을 생성하고 돌릴지 결정 
* llm으로 만드는 알파들을 따로 json으로 저장
* csv 로드하고 저장하는 함수

In [9]:
# 유저카운트와 필드카운트로 몇번 expression을 생성하고 돌릴지 결정 
# 한번에 100개씩 generate하니까 10은 알파를 1000개 만들어서 돌리겠다는 뜻
def datset_to_simnum(dataset):
    score = int(np.log10(datasets_dict[dataset]['userCount']+1) + datasets_dict[dataset]['fieldCount'])

    if score <= 3:
        return 1
    elif score <= 10:
        return 2
    elif score <= 50:
        return 3
    elif score <= 100:
        return 10
    else:
        return 20


# llm으로 알파 리스트를 만들면 json도 저장해주기
def get_json_num(dataset):

    json_list = [x for x in os.listdir('./gen_json/') if dataset in x]
    if len(json_list) == 0:
        return 0
    nums = [int(x.split('_')[-1].split('.')[0]) for x in json_list]

    return max(nums)+1

# csv 로드하고 저장하는 함수
def load_csv(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame()
    df = pd.read_csv(path) # index_col=None(기본값): 인덱스 컬럼으로 읽지 않음
    # 과거에 인덱스가 저장되어 있으면 'Unnamed: 0' 같은 컬럼 제거
    df = df.loc[:, ~df.columns.str.match(r'^Unnamed: \d+$')]
    return df

def save_csv(df: pd.DataFrame, path: str) -> None:
    df.to_csv(path, index=False)

def append_and_save_csv(path: str, new_rows: pd.DataFrame) -> pd.DataFrame:
    base = load_csv(path)
    out = pd.concat([base, new_rows], ignore_index=True) # 인덱스 재부여
    save_csv(out, path) # index=False
    return out

# 시뮬레이션 돌리기
* 데이터셋에 따라 정해진 횟수에 맞추어 시뮬레이션 돌리기
* 원한다면 datasets_list를 조작하여 원하는 데이터셋에 맞추어 돌릴 수 있음

In [ ]:
for dataset in datasets_list[1:]:

        simnum = datset_to_simnum(dataset)

        for i in range(simnum):

            print(f"---working on {dataset}---")
            dataset_category = datasets_dict[dataset]['category_id']

            batch_name = f"{region}_{universe}_{dataset_category}"

            # 리스트로 받아오기
            print("---generating alphas with llm---")
            try:
                # LLM으로 알파 생성해서 response로 받기 -> 정확한 프롬프트와 생성 방법은 generate_expressions_from_dataset() 참조
                response = llm.generate_expressions_from_dataset(s, region, universe, dataset, model = 'gpt-5-mini-2025-08-07', datafields_num_cap = 500 )
            except:
                continue

            print("---received response---")
            json_response = llm.cut_first_to_last_brace(response)
            alphas = json.loads(json_response)
            print(f"---alpha gened well---")

            # 일단 json 저장
            llm.save_json(alphas, f"./gen_json/{dataset}_{get_json_num(dataset)}.json")

            # 리스트로 돌리기 (데이터셋의 score에 따른) 횟수만큼


            print(f"---gened alpha batch {i}---")

            # 미리 sanity check를 하고 안돌아가는 불량 알파들을 사전에 걸러내기
            # 참고로 sanity checker는 완벽하지 않습니다...ㅜㅜ
            # 불완전한 오퍼레이터가 있다면 operators_list.json을 수정해보세요.
            alpha_expressions = [x for x in alphas["results"] if sanity_checker(x['implementation'], _debug=False)]

            alpha_list = [ace.generate_alpha(regular =x['implementation'], region = region, universe = universe, neutralization="INDUSTRY") for x in alpha_expressions]

            len_alpha_list = len(alpha_list)

            print(f"eligible_alpha_count: {len_alpha_list}")

            tags_list = [['llm_test', f"confidence_{x['confidence_level']}"] for x in alpha_expressions]

            descs_list = [x['description'] for x in alpha_expressions]



            for rep in list(range(0,len_alpha_list,8)):

                # 세션이 거의 다되면 새로운 세션 만들기
                if ace.check_session_timeout(s) < 500:
                    s = ace.start_session()
                try:
                    print(f"---simulating alpha batch {rep} to {min(rep+8,len_alpha_list)}---")

                    # 한 번에 8개씩 돌리기
                    sim_results = list(
                            ace.multi_simulate_alphas_map(s,    alpha_list[rep:min(rep+8,len_alpha_list)],
                                                                tags_list[rep:min(rep+8,len_alpha_list)],
                                                                descs_list[rep:min(rep+8,len_alpha_list)],   min(rep+8,len_alpha_list)-rep )

                            )

                    df = pd.DataFrame([ace.flatten_dict(x) for x in sim_results if 'id' in x.keys()])

                    # 시뮬레이션 결과 CSV로 저장
                    append_and_save_csv(f"./results/{batch_name}.csv", df)
                except Exception as e:
                    print(e)

---working on shortinterest43---
---generating alphas with llm---
{
  "results": [
    {
      "idea": "BATSH short-volume share cross-sectional zscore",
      "description": "Uses BATSH short volume share (short_vol / total_vol) as cross-section mean-reversion signal. Applied SUGGESTION1 by combining two datafields (svol and totalvolume). SUGGESTION2 not applied (no vector fields in these data). SUGGESTION3 not applied because both fields have coverage = 1.0. SUGGESTION4 applied: selected BATSH totalvolume (userCount=69, alphaCount=102) to favor well-used fields. Confidence set to 0.65 based on economic rationale (shorting pressure -> reversal) and data robustness.",
      "implementation": "zscore(ts_zscore(divide(shrt43_batsshvol_svol, shrt43_batsshvol_totalvolume), 5))",
      "confidence_level": 0.65
    },
    {
      "idea": "BATSH short-volume momentum (5d returns)",
      "description": "Momentum in short-volume share: recent increase in short-share may predict continued outpe

# LLM 함수 해설

In [ ]:
import random
# def generate_expressions_from_dataset(s, alpha_region, alpha_universe, dataset_id, model = 'gpt-5-mini-2025-08-07', datafields_num_cap = 500, alpha_num = 100 ):

# 함수 인풋들
alpha_region = "USA"
alpha_universe = "TOP3000"
dataset_id = "other463"
datafields_num_cap = 500
alpha_num = 100

# 데이터셋에서 데이터필드 불러오기
data_fields = ace.get_datafields(s, region=alpha_region, universe=alpha_universe, dataset_id=dataset_id, data_type='ALL')
data_fields = data_fields[data_fields['type'] != "GROUP"]
# data_fields는 [[id1, id2, id3, ...], [description1, description2, description3, ...]] 이런 형태로 생김

# data_fileds를 각각의 아이템으로 변경 
data_fields_list = [row.to_dict() for row in data_fields.iloc]

# 데이터필드 리스트에서 필요한 것들만 불러와서 토큰 줄이기
keys = ['id','description','subcategory','coverage','userCount','alphaCount']
data_fields_inputs = [{k:d[k] for k in keys if k in d} for d in data_fields_list]


# 데이터필드가 너무 많을 경우 적당히 개수에 캡을 둘 것
if len(data_fields_inputs) > datafields_num_cap:
    data_fields_inputs = random.sample(data_fields_inputs, datafields_num_cap)



# 오퍼레이터도 같은 방식으로 만들어 줌
operators = ace.get_operators(s)
operators_list = [row.to_dict() for row in operators[operators['scope']=='REGULAR'].iloc]

# rank는 LLM이 너무 많이 씀 - 빼줌
operator_exclude = ['rank']
operators_list = [x for x in operators_list if x['name'] not in operator_exclude]


answer_form = '''{
    "results":[
        {
            "idea":...,
            "description":...,
            "implementation":...,
            "confidence_level":...
        }
    ]
}'''

prompt = f"""
<MISSION>
Based on the OPERATORS and DATA below,
1. SUGGEST {alpha_num} SEPARATED and DIVERSIFIED Alpha ideas which can create excess return in the market
2. Divide your idea into 4 parts, "idea", "description", "implementation", and "target".
3. "idea" must contain the core idea of the alpha.
4. "description" must include how you considered SUGGESTIONS and KEEP_IN_MINDs, and how much you set the confidence of this alpha. Consider SUGGESTIONS and write the reason you applied or didn't apply each of the SUGGESTIONS.
5. "implementation" must contain implementation of the variation, including specific template (in OPERATOR(MATRIX) form) or exact name of datafield.
6. "confidence_level" should be numerical value set in description, in 0-1 scale. Bigger value means higher confidence of this alpha idea.
Very new user can understand the idea and how to implement this idea.
</MISSION>

<SUGGESTIONS>
SUGGESTION1: Actively use MULTIPLE datafields in DATA. Your main mission is finding great combinations of datafields, among infinite combiations of datafields.
SUGGESTION2: Identify VECTOR type datafiled and wrap it with vec_avg() or vec_sum() operator.
SUGGESTION3: If the datafield's coverage is lower than 0.6, try using ts_backfill() to preprocess the data.
SUGGESTION4: datafields' userCount and alphaCount are the count of users and alphas submitted. Try to use high user and alphaCount so that you can catch the signal easily.
</SUGGESTIONS>

<KEEP_IN_MIND>
KEEP_IN_MIND1: Final implementation MUST NOT be too long.
KEEP_IN_MIND2: Final implementation MUST NOT contain over 7 operators and over 2 datafields.
KEEP_IN_MIND3: You CANNOT use datafiled outside of the datafield lists
KEEP_IN_MIND4: You CANNOT use type=GROUP field by itself. You need to use it as “group” parameter in Group operator.
</KEEP_IN_MIND>

<OPERATORS>
You can use those operators: {operators_list}
</OPERATORS>

<DATA>
And data: {data_fields_inputs}
</DATA>

<ANSWER_FORMAT>
You must answer in this form 
{answer_form}
</ANSWER_FORMAT>
""".strip()


print(prompt)

<MISSION>
Based on the OPERATORS and DATA below,
1. SUGGEST 100 SEPARATED and DIVERSIFIED Alpha ideas which can create excess return in the market
2. Divide your idea into 4 parts, "idea", "description", "implementation", and "target".
3. "idea" must contain the core idea of the alpha.
4. "description" must include how you considered SUGGESTIONS and KEEP_IN_MINDs, and how much you set the confidence of this alpha. Consider SUGGESTIONS and write the reason you applied or didn't apply each of the SUGGESTIONS.
5. "implementation" must contain implementation of the variation, including specific template (in OPERATOR(MATRIX) form) or exact name of datafield.
6. "confidence_level" should be numerical value set in description, in 0-1 scale. Bigger value means higher confidence of this alpha idea.
Very new user can understand the idea and how to implement this idea.
</MISSION>

<SUGGESTIONS>
SUGGESTION1: Actively use MULTIPLE datafields in DATA. Your main mission is finding great combinations 